# Importando bibliotecas 

Importamos las bibliotecas necesarias para que funcione el notebook.

In [ ]:
import pandas as pd
from pandas import DataFrame
from pandas import Series
import nltk
from nltk import RegexpTokenizer
from nltk.corpus import stopwords
import json
import numpy as np
from numpy import linalg as LA
from sklearn import decomposition
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from random import choice
from operator import itemgetter
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from IPython.display import clear_output
import warnings
warnings.filterwarnings('ignore')

# Creación del dataframe de las cartas

Creamos el dataframe de las cartas del juego a partir del archivo 'cards.json'.

In [ ]:
dictionary = {}
#Carga del json
with open('cards.json') as f:
    dictionary = json.load(f)
#Creamos un dataframe que contenga todas las cartas
black_cards = DataFrame.from_dict(dictionary['blackCards'])
white_cards = DataFrame.from_dict(dictionary['whiteCards'])
black_cards['type'] = ['prompt' for i in range(len(black_cards))]
black_cards['tags'] = [[] for i in range(len(black_cards))]
white_cards['type'] = ['response' for i in range(len(white_cards))]
black_cards = black_cards[['text','type','tags']]
white_cards = white_cards[['text','type','tags']]
cards = black_cards.append(white_cards, sort = False)
cards = cards.reset_index(drop = True)
cards

# Carga de vectores Word2Vec

Cargamos los vectores word2vec, cada palabra es representada por un vector de 50 dimensiones.

In [ ]:
with open('card_vectors.json') as f:
    word_vectors = json.load(f)

# Creación de vectores promedio para cada carta

Función que crea un vector promedio para cada carta.

Creamos un vector promedio para cada carta omitiendo las palabras que no nos otorgan ningún tipo de información (stopwords). En el caso de las cartas blancas, también promediamos los vectores correspondientes a sus etiquetas.

Existen tres tipos de vectores promedio: 'AverageVector', 'NNVBVector' y 'WeightedAvgVector'.

In [ ]:
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
#vector_type: tipo de vector
#cards: dataframe de las cartas
def vectors(vector_type, cards):
    tokenizer = RegexpTokenizer(r'\w+')
    #Diccionario con listas de vectores y lista con vectores promedio para el dataframe
    c_vectors, values = {}, []
    for i in cards.index.values:
        text = str(cards['text'].loc[i].encode('utf8'))
        c_vectors[i] = []
        for word in tokenizer.tokenize(text):
            #Omitimos las stop words y nos aseguramos que este en nuestros vectores word2vec
            if word.lower() in word_vectors.keys() and word.lower() not in stopwords.words('english'):
                if vector_type == 'AverageVector':
                    c_vectors[i].append(np.array(word_vectors[word.lower()]))
                elif vector_type == 'NNVBVector':
                    if nltk.pos_tag(word.lower())[0][1] == 'VB' or nltk.pos_tag(word.lower())[0][1] == 'NN':
                        c_vectors[i].append(np.array(word_vectors[word.lower()]))
                else:
                    if nltk.pos_tag(word.lower())[0][1] == 'VB' or nltk.pos_tag(word.lower())[0][1] == 'NN':
                        c_vectors[i].append(np.array(word_vectors[word.lower()]) * 1.5)
                    else:
                        c_vectors[i].append(np.array(word_vectors[word.lower()]))
        #Agregamos los vectores de las etiquetas
        if cards['tags'].loc[i] != []:
            for word in cards['tags'].loc[i]:
                if word.lower() in word_vectors.keys() and word.lower() not in stopwords.words('english'):
                    if vector_type == 'WeightedAvgVector':
                        c_vectors[i].append(np.array(word_vectors[word.lower()]) * 1.5)
                    else:
                        c_vectors[i].append(np.array(word_vectors[word.lower()]))
    #Sumamos los vectores y calculamos el promedio de estos
    for i in cards.index.values:
        sum_arr = np.zeros(50)
        for arr in c_vectors[i]:
            sum_arr += arr
        if len(c_vectors[i]) == 0:
            n = 1
        else:
            n = len(c_vectors[i])
        avg_arr = []
        for s in sum_arr:
            avg_arr.append(s/n)
        values.append(np.array(avg_arr))
    #Creamos la nueva columna
    cards[vector_type] = values
    return cards

# Creación de vectores promedio

Creamos un vector promedio para cada carta.

In [ ]:
cards = vectors('AverageVector', cards)
cards

# Creación de vectores promedio utilizando solamente verbos y sustantivos

Creamos otro vector promedio para cada carta tomando en cuenta solamente verbos y sustantivos.

In [ ]:
cards = vectors('NNVBVector', cards)
cards

# Creación de vectores promedio con pesos de 150% para verbos y sustantivos

Creamos un último vector promedio para cada carta dándole un peso de 150% a los verbos y sustantivos. En el caso de las cartas blancas, este peso también se ve reflejado en sus etiquetas.

In [ ]:
cards = vectors('WeightedAvgVector', cards)
cards

# Analizando las etiquetas de las tarjetas blancas

Realizamos un análisis simple de las etiquetas que tienen las cartas blancas.

In [ ]:
all_tags = []
for i in range(len(cards)):
    for tag in cards.loc[i]['tags']:
        all_tags.append(tag)
Series(all_tags).value_counts()

# Creación de una lista de arreglos para graficación

Separamos los vectores promedio de las cartas blancas y negras.

In [ ]:
#Vectores de cartas negras
prompt_vecs = []
for arr in cards[cards['type'] == 'prompt']['WeightedAvgVector'].values:
    prompt_vecs.append(arr)
#Vectores de cartas blancas
response_vecs = []
for arr in cards[cards['type'] == 'response']['WeightedAvgVector'].values:
    response_vecs.append(arr)

# Utilización de PCA para reducir dimensiones

Función para reducir la dimensionalidad de los vectores utilizando PCA.

Toma una serie de vectores y reduce su dimensionalidad.

In [ ]:
#vectors: vectores de entrada
#num: número de dimensiones a reducir
def PCA(vectors, num):
    pca = decomposition.PCA(n_components = num)
    pca.fit(vectors)
    reduced_vectors = pca.transform(vectors)
    return (reduced_vectors)

# Graficación en 3 y 2 dimensiones

Reducimos la dimensionalidad de los vectores y los graficamos en 2 y 3 dimensiones.

In [ ]:
#Pasamos de 50 dimensiones a 3
prompt_vecs3 = PCA(prompt_vecs, 3)
response_vecs3 = PCA(response_vecs, 3)
#Pasamos de 50 dimensiones a 2
prompt_vecs2 = PCA(prompt_vecs, 2)
response_vecs2 = PCA(response_vecs, 2)
#Creación de figura
fig = plt.figure(figsize = [20,10])
ax3 = fig.add_subplot(121, projection='3d')
ax2 = fig.add_subplot(122)
#Colocamos los puntos en el espacio y les damos formato
for point in prompt_vecs3:
    ax3.scatter(point[0], point[1], point[2], s=10, c='r', marker='x')
for point in response_vecs3:
    ax3.scatter(point[0], point[1], point[2], s=10, c='c', marker='o')
for point in prompt_vecs2:
    ax2.scatter(point[0], point[1], s=10, c='r', marker='x')
for point in response_vecs2:
    ax2.scatter(point[0], point[1], s=10, c='c', marker='o')
#Etiquetas de los ejes
ax3.set_xlabel('X')
ax3.set_ylabel('Y')
ax3.set_zlabel('Z')
ax2.set_xlabel('X')
ax2.set_ylabel('Y')
plt.show()

# Clusterización con K-Medias

Utilizamos K-Medias para realizar un clustering de los vectores de las cartas.

In [ ]:
#Clustering para graficar
#Datos para graficar
plot_data = cards[['text','type','WeightedAvgVector']]
#Obtenemos los vectores con pesos
X = plot_data['WeightedAvgVector'].values.tolist()
kmeans = KMeans(n_clusters = 15, random_state = 0, n_init = 100, max_iter = 500).fit_predict(X)
plot_data['cluster'] = kmeans

# Creando coordenadas para T-SNE

De nuevo reducimos dimensionalidad, ahora utilizando T-SNE y generamos las coordenadas para su posterior graficación.

In [ ]:
#Obtenemos los vectores con pesos
X = plot_data['WeightedAvgVector'].values.tolist()
print('Training TSNE model...')
model = TSNE(n_components = 2, random_state = 0, perplexity = 15)
coords = model.fit_transform(X)
x_coords = [coord[0] for coord in coords]
y_coords = [coord[1] for coord in coords]
#Coordenadas
plot_data['TSNE_X'] = x_coords
plot_data['TSNE_Y'] = y_coords
print('Done.')
x_max = plot_data['TSNE_X'].max()
y_max = plot_data['TSNE_Y'].max()
x_min = plot_data['TSNE_X'].min()
y_min = plot_data['TSNE_Y'].min()

# Graficación de T-SNE

Graficación de los vectores en dos dimensiones, utilizando Bokeh. Agregamos etiquetas y colores distintos a cada cluster.

In [ ]:
from bokeh.plotting import *
from bokeh.models import *
palette = ['#1f77b4', '#aec7e8', '#ff7f0e', '#ffbb78', '#2ca02c', '#98df8a', '#d62728', '#ff9896', '#9467bd', '#c5b0d5',
           '#8c564b', '#c49c94', '#e377c2', '#f7b6d2', '#7f7f7f', '#c7c7c7', '#bcbd22', '#dbdb8d', '#17becf', 
           '#9edae5']
cluster_colors = [palette[i] for i in range(10)]
hover = HoverTool(
    tooltips = [
        ('text', '@desc'),
        ('Q/A', '@qa')
    ]
)
tools = [hover, PanTool(), ResetTool()]
output_file('CAH_TSNE.html')
p = figure(plot_width = 1000, plot_height = 1000, tools = tools, title = 'CAH Cards',
           x_range = (x_min-1,x_max+1), y_range = (y_min-1,y_max+1))
plot_data.sort_values(by = 'cluster', inplace = True)
indices = plot_data.index.tolist()
print('Building plot...')
num_plotted = 0
for i in range(15):
    print('Cluster: {}'.format(i))
    prompt_x_list = []
    prompt_y_list = []
    resp_x_list = []
    resp_y_list = []
    prompt_desc_list = []
    resp_desc_list = []
    p_QA_list = []
    r_QA_list = []
    while len(indices) > 0 and plot_data.loc[indices[0]]['cluster'] == i:
        r = indices[0]
        if plot_data.loc[r]['type'] == 'prompt':
            prompt_x_list.append(plot_data.loc[r]['TSNE_X'])
            prompt_y_list.append(plot_data.loc[r]['TSNE_Y'])
            prompt_desc_list.append(plot_data.loc[r]['text'])
            p_QA_list.append('Prompt')
        else:
            resp_x_list.append(plot_data.loc[r]['TSNE_X'])
            resp_y_list.append(plot_data.loc[r]['TSNE_Y'])
            resp_desc_list.append(plot_data.loc[r]['text'])
            r_QA_list.append('Response')
        indices.pop(0)
        num_plotted +=1
    p_source = ColumnDataSource(
        data = dict(
            x = prompt_x_list,
            y = prompt_y_list,
            desc = prompt_desc_list,
            qa = p_QA_list
        )
    )
    r_source = ColumnDataSource(
        data = dict(
            x = resp_x_list,
            y = resp_y_list,
            desc = resp_desc_list,
            qa = r_QA_list
        )
    )
    p.triangle('x', 'y', size = 20, source = p_source, color = palette[i], alpha = .8)
    p.circle('x', 'y', size = 10, source = r_source, color = palette[i], alpha = .4)
    print('Row: {}'.format(num_plotted))
show(p)

# ¿Cuáles son las mejores respuestas? (Distancia euclidiana)

Función que obtiene las mejores cartas blancas para cada carta negra.

Calcula la distancia euclidiana entre los vectores, en este caso, entre menor sea la distancia entre dos vectores, mayor es su relación.

In [ ]:
#cards: dataframe de las cartas
#vector_type: tipo de vector
#num: número de cartas blancas a generar
def best_response_euclidian(cards, vector_type, num):
    df = DataFrame(columns = ['black card'] + [str(i) for i in range(1,num + 1)])
    for i in cards[cards['type'] == 'prompt'].index.values:
        prompt = cards.iloc[i]
        #Vector de carta negra
        prompt_vec = prompt[vector_type]
        response_distances, aux_row = [], [prompt['text']]
        for j in cards[cards['type'] == 'response'].index.values:
            response = cards.iloc[j]
            #Vector de carta blanca
            response_vec = response[vector_type]
            #Distancia euclidiana (Norma de la resta)
            distance = abs(LA.norm(prompt_vec - response_vec))
            response_distances.append([distance, j])
        #Ordenamos respecto a las distancias
        response_distances = sorted(response_distances, key = itemgetter(0))
        #Recortamos la lista, solo tomamos los elementos que nos interesan
        response_distances = response_distances[0:num]
        for d in response_distances:
            aux_row.append(cards.iloc[d[1]]['text'])
        df_aux = DataFrame([aux_row], columns = ['black card'] + [str(i) for i in range(1,num + 1)])
        df = df.append(df_aux, ignore_index = True)
    df = df.reset_index(drop = True)
    return df

# Mejores respuestas (Vector promedio)

In [ ]:
euclidian_average = best_response_euclidian(cards, 'AverageVector', 3)
euclidian_average.loc[0:10]

# Mejores respuestas (Solamente verbos y sustantivos)

In [ ]:
euclidian_nnvb = best_response_euclidian(cards, 'NNVBVector', 3)
euclidian_nnvb.loc[0:10]

# Mejores respuestas (Verbos y sustantivos con 150% peso)

In [ ]:
euclidian_wv = best_response_euclidian(cards, 'WeightedAvgVector', 3)
euclidian_wv.loc[0:10]

# ¿Cuáles son las mejores respuestas? (Simulitud de coseno)

Función que obtiene las mejores cartas blancas para cada carta negra.

Calcula la similitud de coseno entre los vectores, en este caso, entre mayor sea el coseno entre dos vectores, mayor es la relación que existe entre ambos, es decir, el ángulo formado entre los dos vectores es cercano a 0.

In [ ]:
#cards: dataframe de las cartas
#vector_type: tipo de vector
#num: número de cartas blancas a generar
#sim: valor de similitud de coseno (por defecto 1)
def best_response_cosine_similarity(cards, vector_type, num, sim = 1):
    df = DataFrame(columns = ['black card'] + [str(i) for i in range(1,num + 1)])
    for i in cards[cards['type'] == 'prompt'].index.values:
        prompt = cards.iloc[i]
        #Vector de carta negra
        prompt_vec = prompt[vector_type]
        response_similarities, aux_row = [], [prompt['text']]
        for j in cards[cards['type'] == 'response'].index.values:
            response = cards.iloc[j]
            #Vector de carta blanca
            response_vec = response[vector_type]
            #Producto punto entre el producto de las normas
            similarity = np.dot(prompt_vec, response_vec) / (LA.norm(prompt_vec) * LA.norm(response_vec))
            if sim == 0:
                similarity = abs(sim - similarity)
            else:
                similarity = abs((sim - similarity)/sim)
            response_similarities.append([similarity, j])
        #Ordenamos respecto a la similitud de coseno
        response_similarities = sorted(response_similarities, key = itemgetter(0))
        #Recortamos la lista, solo tomamos los elementos que necesitamos
        response_similarities = response_similarities[0:num]
        for s in response_similarities:
            aux_row.append(cards.iloc[s[1]]['text'])
        df_aux = DataFrame([aux_row], columns = ['black card'] + [str(i) for i in range(1,num + 1)])
        df = df.append(df_aux, ignore_index = True)
    df = df.reset_index(drop = True)
    return df

# Mejores respuestas (Vector promedio)

In [ ]:
cosine_average = best_response_cosine_similarity(cards, 'AverageVector', 3)
cosine_average.loc[0:10]

# Mejores respuestas (Solamente verbos y sustantivos)

In [ ]:
cosine_nnvb = best_response_cosine_similarity(cards, 'NNVBVector', 3)
cosine_nnvb.loc[0:10]

# Mejores respuestas (Verbos y sustantivos con 150% peso)

In [ ]:
cosine_wv = best_response_cosine_similarity(cards, 'WeightedAvgVector', 3)
cosine_wv.loc[0:10]

# Ajustándose al usuario

Función que 'ajusta' la máquina al humor del usuario.

Genera una similitud de coseno promedio que se ajusta a tus elecciones. El humor es subjetivo y cada persona tiene su propia visión de este, por lo que ajustar la computadora a tu comportamiento puede dar mejores resultados.

In [ ]:
#cards: dataframe de las cartas
#vector_type: tipo de vector
#num_r: número de rondas a jugar
import time
def train_ia(cards, vector_type, num_r):
    avg_user = []
    for t in range(num_r):
        clear_output()
        white_cards_user = []
        #Carta negra random
        r = choice(cards[cards['type'] == 'prompt'].index.values.tolist())
        black_card = cards.iloc[r]
        c = black_card['text'].count('_')
        if c == 0:
            c = 1
        prompt_vec = black_card[vector_type]
        #5 cartas blancas random
        for i in range(10):
            n = choice(cards[cards['type'] == 'response'].index.values.tolist())
            #Cartas blancas para el usuario
            response = cards.iloc[n]
            response_vec = response[vector_type]
            similarity = np.dot(prompt_vec, response_vec) / (LA.norm(prompt_vec) * LA.norm(response_vec))
            white_cards_user.append([response['text'], similarity])
        print('Round {}'.format(t + 1))
        print('Black Card\n{}'.format(black_card['text']))
        print('White Cards:')
        for w in range(len(white_cards_user)):
            print('{}. {}'.format(w + 1, white_cards_user[w][0]))
        e = input('Choose {} White Cards: (Ej. 1 2 3): '.format(c))
        e = e.split(' ')
        print('Your choices:')
        for w in e:
            print(white_cards_user[int(w) - 1][0])
        print('Round finished\n')
        a = 0
        for j in range(c):
            a += white_cards_user[int(e[j]) - 1][1]
        avg_user.append(1/(a/c))
        time.sleep(3)
    #Media ármonica
    return num_r/sum(avg_user)

In [ ]:
sim_user = train_ia(cards, 'WeightedAvgVector', 15)
sim_user

# Mejores respuestas ajustadas al usuario

In [ ]:
best_user = best_response_cosine_similarity(cards, 'WeightedAvgVector', 3, sim_user)
best_user.loc[0:10]

# La computadora versus el usuario

Función que simula rondas del juego.

Una carta negra es desplegada y tanto el usuario como la computadora deben elegir las mejores cartas blancas.

In [ ]:
#cards: dataframe de las cartas
#vector_type: tipo de vector
#num_r: Número de rondas a jugar
#mode: modo de juego ('Different' para que la IA y el usuario tengan diferentes cartas, 'Equal' para que tengan las
#mismas cartas)
#sim: similitud de coseno a comparar (por defecto 1)
import time
def cah(cards, vector_type, num_r, mode, sim = 1):
    for t in range(num_r):
        clear_output()
        white_cards_ia, white_cards_user = [], []
        #Carta negra random
        r = choice(cards[cards['type'] == 'prompt'].index.values.tolist())
        black_card = cards.iloc[r]
        c = black_card['text'].count('_')
        if c == 0:
            c = 1
        prompt_vec = black_card[vector_type]
        #10 cartas blancas random
        for i in range(10):
            n = choice(cards[cards['type'] == 'response'].index.values.tolist())
            #Cartas blancas de la IA
            response = cards.iloc[n]
            response_vec = response[vector_type]
            similarity = np.dot(prompt_vec, response_vec) / (LA.norm(prompt_vec) * LA.norm(response_vec))
            if sim == 0:
                similarity = abs(sim - similarity)
            else:
                similarity = abs((sim - similarity)/sim)
            white_cards_ia.append([response['text'], similarity])
            if mode == 'Different':
                #Cartas blancas para el usuario
                m = choice(cards[cards['type'] == 'response'].index.values.tolist())
                response = cards.iloc[m]
                response_vec = response[vector_type]
                similarity = np.dot(prompt_vec, response_vec) / (LA.norm(prompt_vec) * LA.norm(response_vec))
                if sim == 0:
                    similarity = abs(sim - similarity)
                else:
                    similarity = abs((sim - similarity)/sim)
            white_cards_user.append([response['text'], similarity])
        white_cards_ia = sorted(white_cards_ia, key = itemgetter(1))
        #IA elige sus cartas blancas para completar la carta negra
        white_cards_ia = white_cards_ia[0:c]
        print('Round {}'.format(t + 1))
        print('Black Card\n{}'.format(black_card['text']))
        print('White Cards:')
        for w in range(len(white_cards_user)):
            print('{}. {}'.format(w + 1, white_cards_user[w][0]))
        e = input('Choose {} White Cards (Ej. 1 2 3): '.format(c))
        e = e.split(' ')
        print('Your choices:')
        for w in e:
            print(white_cards_user[int(w) - 1][0])
        print('IA choices:')
        for w in white_cards_ia:
            print(w[0])
        print('Round finished\n')
        time.sleep(7)
    return

# La computadora con las mismas cartas del usuario

In [ ]:
cah(cards, 'WeightedAvgVector', 10, 'Equal', sim_user)

# La computadora y el usuario con cartas distintas

In [ ]:
cah(cards, 'WeightedAvgVector', 10, 'Different', sim_user)

# La computadora y el usuario con las mismas cartas (el coseno de 90 grados)

No está de más experimentar. Para el humor, la respuesta no debe ser 100% literal, sino que debe tener un componente de sorpresa. ¿Y si tomamos el vector "más perpendicular"?

In [ ]:
cah(cards, 'WeightedAvgVector', 10, 'Equal', 0)

# La computadora y el usuario con cartas distintas (vectores paralelos)

In [ ]:
cah(cards, 'WeightedAvgVector', 10, 'Different', 1)